# Reprojecting Data


## Estimating UTM


When working with cities or local areas, it's often important to reproject geographic data into a UTM zone.  
Why? Because geographic coordinates (latitude and longitude) are not measured in consistent distances — one degree of longitude can mean very different lengths depending on how far you are from the equator.

UTM (Universal Transverse Mercator) zones use meters instead of degrees, which makes distance, area, and direction calculations much more accurate for local-scale analysis — like city planning, infrastructure, or urban mapping.


You can calculate a UTM zone yourself based on a point's coordinates.  
To find the **zone number**, you only need the **longitude**.  
The **latitude** will tell you whether the point is in the **northern** or **southern** h


In [ ]:
def utm_zone(longitude):
    zone = int((longitude + 180) / 6) + 1
    return zone

**What this function does:**

- It calculates the **UTM zone number** based on a given **longitude**.
- The Earth is divided into **60 UTM zones**, each 6 degrees of longitude wide.
- The formula `int((longitude + 180) / 6) + 1` shifts the longitude range from `[-180, +180]` to `[0, 360]` and then divides it by 6 to find out which zone it belongs to.
- The result is rounded down using `int(...)` and adjusted by `+1` because UTM zones are numbered starting from 1 (not 0).


Let’s use our function to check the UTM zone number for a location at 30.33 degrees east longitude.


In [ ]:
longitude = 30.33  
zone = utm_zone(longitude)
print("Номер зоны UTM:", zone)

Номер зоны UTM: 36


Actually, the function above was just a fun way to practice writing code — there's a simpler way to solve this task :)

If you're working with a dataset, you can easily find the appropriate UTM zone using the `.estimate_utm_crs()` method.


In [ ]:
admin_border.estimate_utm_crs()

<Derived Projected CRS: EPSG:32636>
Name: WGS 84 / UTM zone 36N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 30°E and 36°E, northern hemisphere between equator and 84°N, onshore and offshore. Belarus. Cyprus. Egypt. Ethiopia. Finland. Israel. Jordan. Kenya. Lebanon. Moldova. Norway. Russian Federation. Saudi Arabia. Sudan. Syria. Turkey. Uganda. Ukraine.
- bounds: (30.0, 0.0, 36.0, 84.0)
Coordinate Operation:
- name: UTM zone 36N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Reprojecting


**Reprojecting** means converting spatial data from one coordinate system to another.  
In practice, it means **recalculating the coordinates** of each point so they fit a different map projection.


But before reprojecting, we need to decide which coordinate system we want to convert to.  
Thanks to the `estimate_utm_crs()` method, we already know the correct CRS for our data.  
Let’s save it to a separate variable so we can use it when reprojecting the dataset.


In [ ]:
data_crs = admin_border.estimate_utm_crs()

To reproject the data in GeoPandas, we use the `.to_crs()` method.  
This function transforms the geometry of the dataset into a new coordinate reference system (CRS).  
We simply pass the target CRS (for example, the one returned by `estimate_utm_crs()`) inside the method.

It recalculates all coordinate values so that the data aligns correctly in the new projection.


In [ ]:
admin_border_utm = admin_border.to_crs(data_crs)
print(admin_border_utm.crs)

EPSG:32636


We’ve successfully reprojected our layer into the appropriate UTM zone.  
Now, our data is in meters instead of degrees, which makes it much more suitable for performing accurate spatial analysis — such as measuring distances, calculating areas, or working with city-scale infrastructure and planning tasks.


## Setting the CRS


### For data without a CRS

Sometimes we encounter spatial data that doesn’t have a defined coordinate reference system (CRS).  
Let’s look at an example using a CSV file with theater locations in Saint Petersburg.

We’ll read the data and create a `GeoDataFrame` from it:


In [ ]:
theaters = pd.read_csv('data/spb_theaters.csv')
theaters[['latitude', 'longitude']] = theaters['coord'].str.split(',', expand=True)
theaters_gdf = gpd.GeoDataFrame(theaters, geometry=gpd.points_from_xy(theaters['longitude'], theaters['latitude']))

Let’s check the coordinate reference system of `theaters_gdf`:


In [ ]:
print(theaters_gdf.crs)

The result is `None` — that's because we didn’t set a CRS when creating the `GeoDataFrame`.  
But sometimes, we simply receive data that comes without a defined CRS.

In such cases, we can assign a CRS using one of two approaches:

- **Option 1:** Use the `.set_crs()` method
- **Option 2:** Directly assign the CRS using the `.crs` attribute


In [ ]:
theaters_gdf = theaters_gdf.set_crs(epsg=4326) # вариант 1

theaters_gdf.crs = "EPSG:4326" # вариант 2

print(theaters_gdf.crs)

### Setting a CRS vs. Reprojecting

**Reprojecting** means converting coordinates from one CRS to another — it changes both the CRS and the actual coordinate values of all geometries in the dataset.

In contrast, when we **set a CRS**, no coordinate transformation takes place. We're simply assigning a CRS label to the existing data, telling GeoPandas how to interpret the coordinates that are already there.

⚠️ It’s important to know what CRS the coordinates are originally in.  
If you assign the wrong CRS, your data might appear in the wrong part of the world — sometimes even on the wrong continent!


## Choosing a Map Projection

One of the most commonly used projections for GIS projects focused on urban or local-scale data is the **Universal Transverse Mercator (UTM)** projection.  
It provides accurate distance and area measurements within each zone, making it ideal for city-level mapping and analysis.

For smaller scales — such as mapping all of Russia or the entire world — choosing the right projection becomes more complex.  
It largely depends on the geographic extent of your data and the specific goals of your project.

A great resource for understanding map projections is the classic 1987 book  
**[Map Projections – A Working Manual](https://pubs.usgs.gov/pp/1395/report.pdf)** 📕.  
It offers detailed explanations of different projection types, their characteristics, history, and practical use cases.


## Summary
